<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/without-finetuning/askApatient_2_pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: certificate has expired
More details here: https://curl.haxx.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-2.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [12]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-2.validation.csv",header=None)
valid.head()

,0,1
0,2,left heel pain
1,4,Discoloration of skin [blotchy] on the neck
2,7,goiter
3,8,arthritis
4,8,arthritis


In [13]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-2.test.csv",header=None)
test.head()

,0,1
0,2,right heel pain
1,4,black and blue marks
2,8,arthritice
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_asapatient_twadr_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
# learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.save_encoder('bigtweet_fold0_first')

In [27]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
# learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.725240,6.088125,0.129288,01:07


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.583079,8.419662,0.215040,01:15


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.983699,8.243212,0.203166,03:18


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.236160,7.062617,0.281003,03:33
1,2.723532,6.238661,0.226913,03:12


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.443193,13.414412,0.079156,03:13
1,2.014417,6.716132,0.240106,03:25


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.807795,10.261972,0.195251,03:10
1,1.617257,8.424730,0.117414,03:23


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.524266,5.996813,0.319261,03:18
1,1.329901,5.324932,0.345646,03:19


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.322825,6.419942,0.278364,03:21
1,1.103116,5.169452,0.299472,03:13


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.104050,4.628075,0.327177,03:25
1,1.078211,3.838098,0.423483,03:27
2,0.829549,2.990288,0.581794,03:28
3,0.777407,2.957280,0.584433,03:32


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.764656,2.832879,0.594987,03:30
1,0.820029,2.920981,0.531662,03:26
2,0.643831,2.325027,0.668865,03:18
3,0.599906,2.183544,0.658311,03:25


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.564636,2.021694,0.684697,03:26
1,0.663616,2.004707,0.687335,03:07
2,0.565563,1.842853,0.715040,03:22
3,0.473041,1.661329,0.744063,03:12


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.493850,1.936803,0.696570,03:14
1,0.544861,1.913271,0.699208,03:27
2,0.493230,1.761835,0.724274,03:23
3,0.388840,1.690578,0.734828,03:28


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.393941,1.878061,0.713720,03:18
1,0.475824,1.688266,0.754617,03:29
2,0.388328,1.482187,0.784960,03:27
3,0.322414,1.475627,0.790237,03:18


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.325558,1.552234,0.771768,03:27
1,0.432082,1.714642,0.753298,03:13
2,0.342332,1.465194,0.788918,03:24
3,0.299494,1.429511,0.796834,03:15


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.319456,1.656520,0.751979,03:14
1,0.367801,1.516004,0.774406,03:21
2,0.303292,1.416200,0.794195,03:14
3,0.255093,1.391488,0.791557,03:20


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [44]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 801, 238, 8, 8, 8, 238, 289, 318, 16, 344, 100, 359, 755, 31, 31, 31, 32, 34, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 39, 623, 719, 666, 49, 775, 49, 49, 775, 775, 775, 775, 775, 775, 775, 810, 157, 53, 53, 53, 53, 37, 58, 60, 62, 62, 775, 434, 70, 70, 70, 71, 711, 71, 289, 76, 289, 79, 79, 265, 79, 79, 111, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 37, 37, 135, 37, 37, 37, 79, 554, 80, 80, 80, 80, 80, 80, 288, 83, 268, 892, 92, 92, 474, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 96, 96, 594, 344, 100, 100, 100, 100, 108, 108, 108, 190, 894, 111, 111, 111, 111, 111, 111, 896, 831, 125, 135, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 289, 100, 132, 135, 135, 135, 137, 139, 142, 143, 719, 151, 151, 152, 714, 157, 161, 598, 163, 166, 166, 172, 172, 172, 172, 172, 172, 190, 746, 180, 182, 182, 182, 182, 183, 183, 337, 187, 187, 187, 187, 187, 189, 190, 190, 190, 190, 190, 190, 190, 190, 193, 193, 194, 197, 19

In [45]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

631
0.7286374133949192
